In [70]:
import pandas as pd 

df = pd.read_csv('df_model_sans_meteo2.csv',index_col=0 )

In [75]:
from sklearn.model_selection import train_test_split

X = df.drop(['box_office_first_week','is_holiday'],axis=1)
y = df['box_office_first_week']
X_train,X_test, y_train,y_test = train_test_split(X,y, train_size=0.9,shuffle=True, random_state=42)

In [72]:
X.columns

Index(['duration', 'nationality', 'views', 'budget', 'season', 'is_holiday',
       'proportion_stars_actors', 'proportion_stars_producers',
       'proportion_stars_director', 'distributor_avg_frequency',
       'genre_action', 'genre_animation', 'genre_arts martiaux',
       'genre_aventure', 'genre_biopic', 'genre_bollywood', 'genre_comédie',
       'genre_comédie dramatique', 'genre_comédie musicale', 'genre_divers',
       'genre_drame', 'genre_epouvante-horreur', 'genre_erotique',
       'genre_espionnage', 'genre_expérimental', 'genre_famille',
       'genre_fantastique', 'genre_guerre', 'genre_historique',
       'genre_judiciaire', 'genre_musical', 'genre_policier', 'genre_péplum',
       'genre_romance', 'genre_science fiction', 'genre_sport event',
       'genre_thriller', 'genre_western'],
      dtype='object')

In [55]:
X.shape

(5611, 38)

In [76]:
# Importer les bibliothèques nécessaires
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import r2_score, mean_squared_error
import optuna
from optuna.samplers import TPESampler
import pandas as pd
import numpy as np

# Définir les paramètres du préprocesseur
numerical_features = ['duration', 'views', 'budget','proportion_stars_actors', 'proportion_stars_producers', 'proportion_stars_director', 'distributor_avg_frequency']
categorical_features = ['nationality', 'season']

numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder="passthrough"
)

best_params = {'n_estimators': 100, 'criterion': 'squared_error', 'max_depth': 10, 'min_samples_split': 3, 'min_samples_leaf': 1, 'max_features': None, 'bootstrap': False}
# Créer le modèle avec les meilleurs hyperparamètres
model = ExtraTreesRegressor(
    n_estimators=best_params['n_estimators'],
    criterion=best_params['criterion'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    max_features=best_params['max_features'],
    bootstrap=best_params['bootstrap'],
    random_state=42
)

pipe = Pipeline([
    ('preprocessor',preprocessor),
    ('model', model)
])



In [77]:
pipe.fit(X_train, y_train)
pipe.score(X_test,y_test)

0.7258761146327493

In [59]:
feature_importance = pipe.named_steps['model'].feature_importances_

all_feature_names = pipe.named_steps['preprocessor'].get_feature_names_out()

# Créer un DataFrame pour visualiser la feature importance
feature_importance_df = pd.DataFrame({'Feature': all_feature_names, 'Importance': feature_importance})

# Trier le DataFrame par ordre décroissant de l'importance
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Afficher la feature importance
display(feature_importance_df)



,Feature,Importance
2,num__budget,0.337496
1,num__views,0.145662
3,num__proportion_stars_actors,0.104293
0,num__duration,0.056370
66,cat__nationality_U.S.A.,0.036524
...,...,...
81,remainder__genre_bollywood,0.000000
32,cat__nationality_Géorgie,0.000000
23,cat__nationality_Croatie,0.000000
45,cat__nationality_Maroc,0.000000
